In [43]:
import os
os.environ["KMP_DUPLICATE_LIB_OK"] = "True"

In [44]:
import torch
import numpy as np
import matplotlib.pyplot as plt
import librosa
from pydub import AudioSegment
import torchaudio
import torchaudio.functional as F
import torch.quantization as quantization
from pydub import AudioSegment
import IPython.display as play
import soundfile as sf

In [54]:
from pydub import AudioSegment

# Load the audio file
audio = AudioSegment.from_file("Pethya_Spanish.m4a", format="m4a")

# Amplify (increase volume by 6 dB)
amplified_audio = audio + 20  # Increase by 6 decibels

# Export with AAC codec
amplified_audio.export("output.m4a", format="mp4", codec="aac")


<_io.BufferedRandom name='output.m4a'>

In [46]:
print("PyTorch version:", torch.__version__)
print("CUDA available:", torch.cuda.is_available())
print("CUDA version:", torch.version.cuda)

PyTorch version: 2.5.1
CUDA available: True
CUDA version: 11.8


In [47]:
from speechbrain.inference.separation import SepformerSeparation as separator
model = separator.from_hparams(source = "speechbrain/sepformer-wsj03mix", savedir = "pretrained_models/sepformer-wsj03mix")

INFO:speechbrain.utils.fetching:Fetch hyperparams.yaml: Fetching from HuggingFace Hub 'speechbrain/sepformer-wsj03mix' if not cached
INFO:speechbrain.utils.fetching:Fetch custom.py: Fetching from HuggingFace Hub 'speechbrain/sepformer-wsj03mix' if not cached
INFO:speechbrain.utils.fetching:Fetch masknet.ckpt: Fetching from HuggingFace Hub 'speechbrain/sepformer-wsj03mix' if not cached
INFO:speechbrain.utils.fetching:Fetch encoder.ckpt: Fetching from HuggingFace Hub 'speechbrain/sepformer-wsj03mix' if not cached
INFO:speechbrain.utils.fetching:Fetch decoder.ckpt: Fetching from HuggingFace Hub 'speechbrain/sepformer-wsj03mix' if not cached
INFO:speechbrain.utils.parameter_transfer:Loading pretrained files for: masknet, encoder, decoder


In [55]:
ip = 'output.m4a'
op = 'MixedSpeech_2.wav'

audio = AudioSegment.from_file(ip,format='mp4')
audio.export(op,format = 'wav')

<_io.BufferedRandom name='MixedSpeech_2.wav'>

In [56]:
torchaudio.set_audio_backend("soundfile")
waveform, sample_rate = torchaudio.load("MixedSpeech_2.wav")

resampled_waveform = F.resample(waveform, sample_rate, 8000)
torchaudio.save("MixedSpeech_2.wav", resampled_waveform, 8000)
print("File resampled to 8 kHz and saved as 'MixedSpeech_2.wav'")

File resampled to 8 kHz and saved as 'MixedSpeech_2.wav'


C:\Users\ashut\AppData\Local\Temp\ipykernel_18328\2579143143.py:1: UserWarning: torchaudio._backend.set_audio_backend has been deprecated. With dispatcher enabled, this function is no-op. You can remove the function call.
  torchaudio.set_audio_backend("soundfile")


In [24]:
import torch
import os

# Save original model weights
torch.save(model.state_dict(), "original_sepformer.pth")

# Compute size of original model
original_size = os.path.getsize("original_sepformer.pth") / (1024 * 1024)  # Convert bytes to MB
print(f"Original Model Size: {original_size:.2f} MB")

# Set model to evaluation mode
model.eval()

# Apply dynamic quantization
quantized_model = torch.quantization.quantize_dynamic(
    model,  # Original model
    {torch.nn.Linear}, 
    dtype=torch.qint8  # Convert FP32 weights to INT8
)

# Save quantized model weights
torch.save(quantized_model.state_dict(), "quantized_sepformer.pth")

# Compute size of quantized model
quantized_size = os.path.getsize("quantized_sepformer.pth") / (1024 * 1024)  # Convert bytes to MB
print(f"Quantized Model Size: {quantized_size:.2f} MB")

# Compute compression ratio
compression_ratio = original_size / quantized_size
print(f"Compression Ratio: {compression_ratio:.2f}x")


Original Model Size: 108.16 MB
Quantized Model Size: 60.21 MB
Compression Ratio: 1.80x


In [57]:
model.eval() #Set the model to evaluation mode

# quantized_model = quantization.quantize_dynamic(
#     model, #original model
#     {torch.nn.Linear}, 
#     dtype = torch.qint8 # Convert the floating point weights to 8-bit integer weights
# )
# torch.save(quantized_model.state_dict(),"quantized_sepformer.pth")
# print("Model Quantized Successfully.")

with torch.no_grad():
    est_sources = model.separate_file(path = "MixedSpeech_2.wav")
torchaudio.save("SQ1.wav", est_sources[:, :, 0].detach().cpu(), 8000)
torchaudio.save("SQ2.wav", est_sources[:, :, 1].detach().cpu(), 8000)
torchaudio.save("SQ3.wav", est_sources[:, :, 2].detach().cpu(), 8000)

In [ ]:
ip = 'SQ2_Reference.m4a'
op = 'SQ2_Reference.wav'

audio = AudioSegment.from_file(ip,format='mp4')
audio.export(op,format = 'wav')

print("Conversion Complete")

In [ ]:
# Load the estimated and reference signals without resampling
y_est, sr_est = librosa.load("SQ2.wav", sr=None)
y_ref, sr_ref = librosa.load("SQ2_Reference.wav", sr=None)

# Ensure the sample rates match
assert sr_est == sr_ref, "Sample rates must be the same!"

# Pad SQ2_Reference to match SQ2
if len(y_ref) < len(y_est):
    pad_width = len(y_est) - len(y_ref)
    y_ref_padded = np.pad(y_ref, (0, pad_width), mode='constant')
else:
    y_ref_padded = y_ref[:len(y_est)]  # Just in case it’s longer

# Save the padded reference back to the same file
sf.write("SQ2_Reference.wav", y_ref_padded, sr_ref)

print(f"Padding complete: SQ2_Reference.wav now has shape {y_ref_padded.shape}")


In [ ]:
y,sr = librosa.load("SQ2.wav")
y.shape

In [ ]:
x,sr = librosa.load("SQ2_Reference.wav")
x.shape

In [ ]:
# Save the quantized model
quantized_model_path = "quantized_sepformer.pth"
torch.save(quantized_model.state_dict(), quantized_model_path)
print(f"Quantized model saved to {quantized_model_path}")

In [ ]:
# Get the size of the quantized model file
file_size_bytes = os.path.getsize(quantized_model_path)
file_size_kb = file_size_bytes / 1024  # Convert to KB
file_size_mb = file_size_kb / 1024  # Convert to MB

print(f"Size of the quantized model: {file_size_mb:.2f} MB")

In [ ]:
def si_snr(reference, estimation, eps=1e-8):
    """
    Compute the Scale-Invariant Signal-to-Noise Ratio (SI-SNR) between a reference
    and an estimated signal.

    Parameters:
        reference (np.ndarray): Clean reference signal.
        estimation (np.ndarray): Estimated signal.
        eps (float): Small constant to avoid division by zero.

    Returns:
        float: SI-SNR value in decibels (dB).
    """
    # Remove the mean (DC component) from both signals
    reference = reference - np.mean(reference)
    estimation = estimation - np.mean(estimation)

    # Compute the scaling factor to align estimation to reference
    scaling_factor = np.dot(estimation, reference) / (np.linalg.norm(reference) ** 2 + eps)
    
    # Projection of the estimated signal on the reference
    s_target = scaling_factor * reference

    # Compute the noise (error) component
    e_noise = estimation - s_target

    # Calculate SI-SNR in dB
    si_snr_value = 10 * np.log10(np.sum(s_target ** 2) / (np.sum(e_noise ** 2) + eps))
    return si_snr_value

# Load the estimated and reference signals without resampling (assumes they are at the same rate)
y_est, sr_est = librosa.load("SQ2.wav", sr=None)
y_ref, sr_ref = librosa.load("SQ2_Reference.wav", sr=None)

# Ensure the sample rates match
assert sr_est == sr_ref, "Sample rates must be the same!"

# Match the signal lengths by trimming the longer one
min_length = min(len(y_est), len(y_ref))
y_est = y_est[:min_length]
y_ref = y_ref[:min_length]

# Calculate SI-SNR
score = si_snr(y_ref, y_est)
print("SI-SNR: {:.2f} dB".format(score))

In [58]:
play.Audio("MixedSpeech_2.wav")

In [61]:
play.Audio("SQ1.wav")

In [62]:
play.Audio("SQ2.wav")

In [ ]:
from pydub import AudioSegment
from pydub.effects import low_pass_filter, high_pass_filter

# Load audio
audio = AudioSegment.from_file("SQ2.wav", format="wav")

# Apply filters
audio = high_pass_filter(audio, cutoff=300)  # Removes low-frequency noise
audio = low_pass_filter(audio, cutoff=3000) # Removes high-frequency noise

# Export cleaned audio
audio.export("SQ2_Filtered.wav", format="wav")

print("Filtered audio saved as output_filtered.wav")


In [ ]:
play.Audio("SQ2_Filtered.wav")

In [ ]:

model = separator.from_hparams(source="speechbrain/sepformer-wham-enhancement", savedir='pretrained_models/sepformer-wham-enhancement')

# for custom file, change path
est_sources = model.separate_file(path='SQ2.wav')
torchaudio.save("SQ2_Filtered.wav", est_sources[:, :, 0].detach().cpu(), 8000)
